Dawit Reda
ID#10189420
Module 2 Acquiring and Storing Data
MCIS 535 - Data Access and Transformation
Submitted to Dr. Daniel Wu

# Requirements

You should have installed MySQL and MongoDB. Write a Python program to import a data file (reference Module 1 if needed) 
and store the imported data to either MySQL or MongoDB. Your program should also show results from querying the stored data 
from your database. Make sure that
1) you debug your code thoroughly and 
2) include both .py file and data file.

In [50]:
#import all important libraries 
from tabulate import tabulate
import pathlib
import pandas as pd
import csv
import MySQLdb
import json
import xml.etree.ElementTree as ET
import mysql.connector
from mysql.connector import errorcode
import xlrd
from openpyxl import  load_workbook
import re
from lxml import etree
import pymysql, os

# Import CSV file to MYSQL

In [51]:
#establish the database connection method
#create try - catch to catch all errors using the errors.Error exception
def connect_to_mysql():
    
    try:
        con = mysql.connector.connect(host='127.0.0.1', user='root', passwd='password')
        
        db_cursor = con.cursor(buffered=True)

        #create a database
        db_cursor.execute('CREATE DATABASE IF NOT EXISTS Dawit_module_2')
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("Error with user name or password")
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("Database does not exist or database name Error")
        else:
            print(err)
    
    return db_cursor, con


connect_to_mysql()

(<mysql.connector.cursor_cext.CMySQLCursorBuffered at 0x1a043edb668>,
 <mysql.connector.connection_cext.CMySQLConnection at 0x1a043edb3c8>)

In [52]:
#show all tables - and check if the database was created
def check_databases():
    cur, conn = connect_to_mysql()
   
    #show all databses 
    cur.execute("SHOW DATABASES")
    for x in cur:
        print(x)
            
            
check_databases()

('dawit_module_2',)
('information_schema',)
('my_first_db',)
('mysql',)
('performance_schema',)
('sakila',)
('sys',)
('world',)


Create a Tables to store data

In [53]:
#create a table to stor the data
def create_tables():
    mycursor, conn = connect_to_mysql()
    conn.connect(database="dawit_module_2")
    
    #table to import csv file
    mycursor.execute('CREATE Table IF NOT EXISTS Payments (id INT AUTO_INCREMENT PRIMARY KEY,Price INT,Payment_Type VARCHAR(255), Name VARCHAR(255) ,City VARCHAR(255))')
    
    #create table to import excel file
    mycursor.execute('CREATE Table IF NOT EXISTS Customers (id INT AUTO_INCREMENT PRIMARY KEY,First_Name VARCHAR(255), Last_Name VARCHAR(255) , Gender VARCHAR(255),Country VARCHAR(255),Age INT,ID_number INT)') 
    #create table to import xml file
    mycursor.execute('CREATE Table IF NOT EXISTS XMLData (id INT AUTO_INCREMENT PRIMARY KEY, customer_name varchar(255), customer_Id varchar(255), phone varchar(255), email varchar(255), address varchar(255))')
    #create table to import JSON file
    mycursor.execute('CREATE TABLE IF NOT EXISTS JsonFile( id int auto_increment primary key, name varchar(255), Version varchar(255), Install varchar(255),Owner varchar(255),Kernel varchar(255) )')

    #mycursor.execute('CREATE Table IF NOT EXISTS JsonData()
    print('Tables Created')
create_tables()   

Tables Created


In [54]:
#Load CSV data into MySQL
def load_CSV_to_MYSQL():
    with open('Module_1.csv') as csvfile:
        reader = csv.DictReader(csvfile, delimiter = ',')
        for row in reader:
            #make connection to database
            cur, conn = connect_to_mysql()
            #select database
            conn.connect(database="dawit_module_2")
            # insert data
            sql_statement = 'INSERT INTO Payments(Price,Payment_Type, Name, City ) VALUES (%s,%s,%s,%s)'
            #excute the sql statments
            cur.executemany(sql_statement,[(row['Price'], row['Payment_Type'], row['Name'], row['City'])])
            #conn.escape_string(sql_statement)
            conn.commit()
        print('File loaded')
            
            
load_CSV_to_MYSQL()

File loaded


In [55]:
#read csv file from MYSQL database
def read_csv_file_from_MYSQL():
    #make connection to database
    cur, conn = connect_to_mysql()
    #select database
    conn.connect(database="dawit_module_2")
    #select all rows from the payment database
    select_statment = """ SELECT * FROM dawit_module_2.payments;"""
    df =pd.read_sql(select_statment, con=conn)
    return df
read_csv_file_from_MYSQL().head()

,id,Price,Payment_Type,Name,City
0,2,1200,Mastercard,carolina,Basildon
1,3,1200,Visa,Betina,Parkville
2,4,1200,Mastercard,Federica e Andrea,Astoria
3,5,1200,Visa,Gouya,Echuca
4,6,3600,Visa,Gerd W,Cahaba Heights


# Import Excel file to MYSQL

In [56]:
#Load excel data into MySQL
def load_Excelfile_to_MYSQL():
    #make connection to database
  
    cur, conn = connect_to_mysql()
    #select database
    conn.connect(database="dawit_module_2")
    # insert data
    book = xlrd.open_workbook('Module_1.xlsx')
    sheet = book.sheet_by_name('customer')
    rows = sheet.nrows
    sql_statement = """INSERT INTO Customers (First_Name, Last_Name , Gender,Country ,Age ,ID_number)
    VALUES (%s,%s,%s,%s,%s,%s)"""
    
    for a in range(1, rows):
        First_Name = sheet.cell(a,0).value
        Last_Name = sheet.cell(a,1).value
        Gender = sheet.cell(a,2).value
        Country = sheet.cell(a,3).value
        Age = sheet.cell(a,4).value
        ID_number = sheet.cell(a,5).value
        
        values = (First_Name, Last_Name , Gender,Country ,Age ,ID_number)
        
        cur.execute(sql_statement,values)
    cur.close()
    conn.commit()
    print('Done')
    
   
        
 #load the data into MYSQl   
load_Excelfile_to_MYSQL()    

Done


In [57]:
#read data from mysql using python
def read_Excelfile_from_MYSQL():

    #make connection to database
    cur, conn = connect_to_mysql()
    #select database
    conn.connect(database="dawit_module_2")
    #select all rows from the payment database
    select_statment = """ SELECT * FROM dawit_module_2.customers;"""
    df =pd.read_sql(select_statment, con=conn)
    
    return df

read_Excelfile_from_MYSQL().head()

,id,First_Name,Last_Name,Gender,Country,Age,ID_number
0,10,Dulce,Abril,Female,United States,32,1562
1,11,Mara,Hashimoto,Female,Great Britain,25,1582
2,12,Philip,Gent,Male,France,36,2587
3,13,Kathleen,Hanner,Female,United States,25,3549
4,14,Nereida,Magwood,Female,United States,58,2468


# Import XML file to MYSQL

In [58]:
#Load xml data into MySQL
def load_XML_to_MYSQL(file):
    #make connection to database
    cur, conn = connect_to_mysql()
    #select database
    conn.connect(database="dawit_module_2")
    # insert data
    sql_statement = """INSERT INTO XMLData (customer_name, customer_Id , phone,email,address )
    VALUES (%s,%s,%s,%s,%s)""" 
    
    document = etree.parse(file)
    for parent in document.findall('customer'):
             
        customer_name = parent.get('name')
        customer_Id = parent.find('ID').text
        phone =parent.find('phone').text
        email = parent.find('email').text
        address = parent.find('address').text

        values = (customer_name, customer_Id , phone,email,address )
        
        cur.execute(sql_statement,values)
        
    cur.close()
    conn.commit()
    print('Done')
  
load_XML_to_MYSQL('Module_1.xml')

Done


In [59]:
#read xml data from mysql using python
def read_xmlData_from_MYSQL():

    #make connection to database
    cur, conn = connect_to_mysql()
    #select database
    conn.connect(database="dawit_module_2")
    #select all rows from the xmldata database
    select_statment = """ SELECT * FROM dawit_module_2.xmldata;"""
    df =pd.read_sql(select_statment, con=conn)
    
    return df
#read the top 5 of the xml data
read_xmlData_from_MYSQL().head()

,id,customer_name,customer_Id,phone,email,address
0,1,gokhan,1423,555-1234,gokhan@gmail.com,Gorman Avenue
1,2,mike,6423,555-6472,mike@gmail.com,Avent ferry Avenue
2,3,john,3323,555-4567,john@gmail.com,Varsity Avenue
3,4,david,1343,555-6472,dawit@gmail.com,Fifth Avenue
4,5,gokhan,1423,555-1234,gokhan@gmail.com,Gorman Avenue


# Import JSON file to MYSQL

In [60]:
#Load JSON data into MySQL
def load_JSON_to_MYSQL():
    #make connection to database
  
    cur, conn = connect_to_mysql()
    #select database
    conn.connect(database="dawit_module_2")
    # insert data
    sql_statement = """INSERT INTO JsonFile (name, Version , Install,Owner,Kernel )
    VALUES (%s,%s,%s,%s,%s)""" 
    
    # read JSON file which is in the next parent folder
    file = os.path.abspath('module_1.json')
    json_data=open(file).read()
    json_object = json.loads(json_data)
    json_files = []
    for a in json_object:
        name = a['Name']
        Version = a['Version']
        Install = a['Install']
        Owner = a['Owner']
        Kernel = a['Kernel']
        
        values = (name, Version , Install,Owner,Kernel )
        
        cur.execute(sql_statement,values)
    cur.close()
    conn.commit()
    print('Done')
 

load_JSON_to_MYSQL()

Done


In [61]:
#read Json data from mysql using python
def read_JSONData_from_MYSQL():

    #make connection to database
    cur, conn = connect_to_mysql()
    #select database
    conn.connect(database="dawit_module_2")
    #select all rows from the xmldata database
    select_statment = """ SELECT * FROM dawit_module_2.jsonfile;"""
    df =pd.read_sql(select_statment, con=conn)
    
    return df
#read the top 5 of the xml data
read_JSONData_from_MYSQL().head()

,id,name,Version,Install,Owner,Kernel
0,1,Debian,9,apt,SPI,4.9
1,2,Ubuntu,17.10,apt,Canonical,4.13
2,3,Fedora,26,dnf,Red Hat,4.13
3,4,CentOS,7,yum,Red Hat,3.10
4,5,OpenSUSE,42.3,zypper,Novell,4.4
